In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Correct file name from the dataset
file_path = "US_Accidents_March23.csv"

# Load dataset into Pandas DataFrame
df = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "sobhanmoosavi/us-accidents",
    file_path,
)



100%|██████████| 653M/653M [00:04<00:00, 160MB/s]

Extracting zip of US_Accidents_March23.csv...


In [ ]:
print("First 5 records:\n", df.head())

First 5 records:
     ID   Source  Severity           Start_Time             End_Time  \
0  A-1  Source2         3  2016-02-08 05:46:00  2016-02-08 11:00:00   
1  A-2  Source2         2  2016-02-08 06:07:59  2016-02-08 06:37:59   
2  A-3  Source2         2  2016-02-08 06:49:27  2016-02-08 07:19:27   
3  A-4  Source2         3  2016-02-08 07:23:34  2016-02-08 07:53:34   
4  A-5  Source2         2  2016-02-08 07:39:07  2016-02-08 08:09:07   

   Start_Lat  Start_Lng  End_Lat  End_Lng  Distance(mi)  ... Roundabout  \
0  39.865147 -84.058723      NaN      NaN          0.01  ...      False   
1  39.928059 -82.831184      NaN      NaN          0.01  ...      False   
2  39.063148 -84.032608      NaN      NaN          0.01  ...      False   
3  39.747753 -84.205582      NaN      NaN          0.01  ...      False   
4  39.627781 -84.188354      NaN      NaN          0.01  ...      False   

  Station   Stop Traffic_Calming Traffic_Signal Turning_Loop Sunrise_Sunset  \
0   False  False         

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
data_dict = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ML practical/us_accidents_data_dictionary.csv")

In [ ]:
list_column = list(data_dict[data_dict['Relevance to Severity']!='Low']['Feature Name'])
print(list_column)
print(len(list_column))

['Severity', 'Start_Time', 'End_Time', 'Distance(mi)', 'Description', 'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition', 'Crossing', 'Give_Way', 'Junction', 'Railway', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']
26


In [ ]:
df = df[list_column]
print(f"Shape of dataframe : {df.shape}")
print(f"Size of dataframe : {df.size}")

Shape of dataframe : (7728394, 26)
Size of dataframe : 200938244


### There are now 26 columns, except those who have low relevancve to severity. Total rows count are 7728394 and size of the data is 200938244

In [ ]:
df.columns

Index(['Severity', 'Start_Time', 'End_Time', 'Distance(mi)', 'Description',
       'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)',
       'Pressure(in)', 'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Weather_Condition', 'Crossing', 'Give_Way',
       'Junction', 'Railway', 'Stop', 'Traffic_Calming', 'Traffic_Signal',
       'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

## Remove **Low Variance** columns from the dataframe

In [ ]:
(df.select_dtypes(include='number')).var()

,0
Severity,0.237687
Distance(mi),3.157056
Temperature(F),361.519011
Wind_Chill(F),501.304566
Humidity(%),520.796565
Pressure(in),1.012418
Visibility(mi),7.227042
Wind_Speed(mph),29.430445
Precipitation(in),0.012149


From variance of all numeric columns, we can clearly see that we can drop some feateures like Pressure, Precipitation from the dataframe which have low variance

In [ ]:
df1 = df.drop(['Pressure(in)', 'Precipitation(in)'], axis=1)
print(df1.shape)

(7728394, 24)


### Finding Variance of Qualitative Datas
### Using Value Counts and Variance

In [ ]:
for col in list((df1.select_dtypes(include='object')).columns) :
  print(f"{col} : {(df1[col].value_counts()).var()}")

Start_Time : 0.8157108258577714
End_Time : 0.29198898133386364
Description : 90.99248305068886
Weather_Timestamp : 350.3647588441413
Wind_Direction : 27169620215.53623
Weather_Condition : 65469205141.59904
Sunrise_Sunset : 4392523512882.0
Civil_Twilight : 6793629744050.0
Nautical_Twilight : 9888633821448.0
Astronomical_Twilight : 12750987401352.0


### It shows that there are two columns Start_Time and End_Time having low variance so will Drop those columns from the dataset

In [ ]:
df1=df1.drop(['Start_Time','End_Time'],axis=1)
print(df1.shape)

(7728394, 22)


### Now there are total 22 columns after dropping Start_Time and End_Time

In [ ]:
df1['Severity'].value_counts()

,count
Severity,
2,6156981
3,1299337
4,204710
1,67366


In [ ]:
df1['Weather_Condition'].value_counts()

,count
Weather_Condition,
Fair,2560802
Mostly Cloudy,1016195
Cloudy,817082
Clear,808743
Partly Cloudy,698972
...,...
Heavy Sleet / Windy,1
Sand / Windy,1
Heavy Rain Shower / Windy,1


## cleaning weather condition along with Severity 2

### checking weather condition of Severity 2

In [ ]:
weather_counts = df1[df1['Severity'] == 2]['Weather_Condition'].value_counts()
weather_counts.head(50)

,count
Weather_Condition,
Fair,2226576
Mostly Cloudy,792735
Cloudy,692929
Partly Cloudy,548760
Clear,536971
Light Rain,270162
Overcast,248938
Scattered Clouds,132748
Light Snow,101703


## Considering only 50000 values or less of every weather condition of Severity 2
### Grouping with Weather Condition and then separating with Seveirity 2 and other reamaining then randomly sampling 50000 of Sevirity 2 and then concatinating with remaining other severity with same weather condition and then did same with other remaining Weather Conditions.

In [ ]:
df_list=[]
for wc, group in df1.groupby('Weather_Condition'):
    sev2 = group[group['Severity'] == 2]
    other = group[group['Severity'] != 2]

    # If more than 60,000 severity 2 rows, sample 60,000, else keep all
    if len(sev2) > 60000:
        sev2 = sev2.sample(60000, random_state=42)

    df_list.append(pd.concat([other, sev2]))

# Combine all weather conditions
df_balanced = pd.concat(df_list).reset_index(drop=True)

In [ ]:
df_balanced.head()

,Severity,Distance(mi),Description,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Visibility(mi),Wind_Direction,Wind_Speed(mph),...,Give_Way,Junction,Railway,Stop,Traffic_Calming,Traffic_Signal,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,3,0.0,Two lanes blocked due to crash on I-15 Northbo...,2022-04-21 18:19:00,74.0,74.0,13.0,5.0,SW,14.0,...,False,False,False,False,False,False,Day,Day,Day,Day
1,3,0.0,#1 lane blocked due to accident on I-405 North...,2021-10-11 18:43:00,65.0,65.0,18.0,4.0,N,20.0,...,False,False,False,False,False,False,Night,Day,Day,Day
2,3,0.0,Left lane blocked due to accident on I-15 Sout...,2021-09-20 05:56:00,50.0,50.0,59.0,10.0,NNW,15.0,...,False,False,False,False,False,False,Night,Night,Night,Day
3,3,0.0,Two lanes blocked due to accident on US-75 Cen...,2021-01-30 18:53:00,70.0,70.0,17.0,4.0,W,9.0,...,False,False,False,False,False,False,Night,Night,Night,Day
4,3,0.0,Accident on Stemmons Fwy Svrd Northbound at In...,2020-10-12 08:53:00,72.0,72.0,41.0,6.0,N,12.0,...,False,False,False,False,False,True,Day,Day,Day,Day


In [ ]:
df_balanced[df_balanced['Severity']==2]['Weather_Condition'].value_counts().head(30)

,count
Weather_Condition,
Clear,60000
Cloudy,60000
Scattered Clouds,60000
Overcast,60000
Mostly Cloudy,60000
Rain,60000
Partly Cloudy,60000
Light Snow,60000
Light Rain,60000


In [ ]:
df_balanced['Severity'].value_counts()

,count
Severity,
3,1269042
2,980246
4,197281
1,66412


### Now we can see that there are only 50000 values per weather condition for Sevirity 2

In [ ]:
df_balanced.shape

(2512981, 22)

## Finding the Null Values in the Dataset

#### Following code result in "Column name", "Datatype", "Number of null values" and "Percentage of Null Values" of columns of the balanced dataset.

In [ ]:
for col, n in df_balanced.isna().sum().items():
    print(col,' '*(25-len(col)),df_balanced[col].dtypes,'\t', n, '\t\t', round((n/len(df_balanced[col])*100),2))

Severity                   int64 	 0 		 0.0
Distance(mi)               float64 	 0 		 0.0
Description                object 	 2 		 0.0
Weather_Timestamp          object 	 0 		 0.0
Temperature(F)             float64 	 9964 		 0.4
Wind_Chill(F)              float64 	 873424 		 34.76
Humidity(%)                float64 	 14100 		 0.56
Visibility(mi)             float64 	 6132 		 0.24
Wind_Direction             object 	 9919 		 0.39
Wind_Speed(mph)            float64 	 195026 		 7.76
Weather_Condition          object 	 0 		 0.0
Crossing                   bool 	 0 		 0.0
Give_Way                   bool 	 0 		 0.0
Junction                   bool 	 0 		 0.0
Railway                    bool 	 0 		 0.0
Stop                       bool 	 0 		 0.0
Traffic_Calming            bool 	 0 		 0.0
Traffic_Signal             bool 	 0 		 0.0
Sunrise_Sunset             object 	 5765 		 0.23
Civil_Twilight             object 	 5765 		 0.23
Nautical_Twilight          object 	 5765 		 0.23
Astronomical_Twilight  

#### Dividing into two part Qualitative Columns and Quantitative Columns

In [ ]:
quanti_col = []; quali_col = [];
for col, n in df_balanced.isna().sum().items():
    if(df_balanced[col].isna().sum()>0):
        if df_balanced[col].dtypes == 'object':
            quali_col.append(col)
        if df_balanced[col].dtypes == 'float64':
            quanti_col.append(col)

In [ ]:
print(quali_col)
print(quanti_col)

['Description', 'Wind_Direction', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']
['Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Visibility(mi)', 'Wind_Speed(mph)']


## Filling the Null Values using SimpleImputer
### where we use two different imputer for two different datatype qualitative and quantitative mean_imputer for Quantitative Data and mode_imputer for Qualitative Data

In [ ]:
# !pip install scikit-learn
from sklearn.impute import SimpleImputer
import numpy as np
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
mode_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

### First we will you mean_imputer to fill Null Values of Quantitative Data

In [ ]:
for col in quanti_col:
    df_balanced[col] = pd.DataFrame(mean_imputer.fit_transform(df_balanced[[col]]))

### After applying the mean imputer lets apply mode imputer ans then check the null values for SUREITY

In [ ]:
for col in quali_col:
    df_balanced[col] = pd.DataFrame(mode_imputer.fit_transform(df_balanced[[col]]))

### Now lets calculate the NULL Values Again to check that are handle or not

In [ ]:
df_balanced.isna().sum()

,0
Severity,0
Distance(mi),0
Description,0
Weather_Timestamp,0
Temperature(F),0
Wind_Chill(F),0
Humidity(%),0
Visibility(mi),0
Wind_Direction,0
Wind_Speed(mph),0


## Finding Duplicates From the Dataset

In [ ]:
df_balanced.duplicated().sum()

np.int64(57365)

### It shows that there are total 19504 duplicate values in the dataset, So lets drop the values

In [ ]:
df_balanced = df_balanced.drop_duplicates()

### Now lets check the dupliocate values again

In [ ]:
df_balanced.duplicated().sum()

np.int64(0)

### Now there are 0 Duplicate Values in the dataframe

### Now check the Final Size and Shape of the DataFrame

In [ ]:
df_balanced.size

54023552

In [ ]:
df_balanced.shape

(2455616, 22)

## Save Balanced Dataframe
### Lets save the Cleaned DataFrame into a CSV File foramt.

In [ ]:
df_balanced.columns

Index(['Severity', 'Distance(mi)', 'Description', 'Weather_Timestamp',
       'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Visibility(mi)',
       'Wind_Direction', 'Wind_Speed(mph)', 'Weather_Condition', 'Crossing',
       'Give_Way', 'Junction', 'Railway', 'Stop', 'Traffic_Calming',
       'Traffic_Signal', 'Sunrise_Sunset', 'Civil_Twilight',
       'Nautical_Twilight', 'Astronomical_Twilight'],
      dtype='object')

In [ ]:
df_balanced.to_csv("cleaned_project_data.csv")